In [79]:
import pandas as pd
from sqlalchemy import *
from create_ref_mariadb import mariadb_connection
from create_ref_mongodb import mongodb_connection
import pymongo
from pprint import pprint
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [48]:
client = mongodb_connection()

def last_forecast_weather_to_df(client):
    # Get the last update
    last_forcast_update = client.forecast.find({}, {"data" : 1}).sort("extract_date", -1).limit(1)
    last_forcast_update = list(last_forcast_update)[0]["data"]

    # Select required data from the document
    list_data = []
    for windfarm in last_forcast_update:
        for forecast in windfarm["list"]:
            dict_forecast = {}
            for key, value in forecast.items():
                dict_forecast[key] = forecast[key]
            dict_forecast["windfarm_id"] = windfarm["windfarm_id"]
            list_data.append(dict_forecast)

    # Transform list in to DataFrame
    dict_rename = {"main.temp": "temp", "main.feels_like": "feels_like", "main.temp_min": "temp_min", "main.temp_max": "temp_max",
                    "main.pressure": "pressure", "main.humidity": "humidity", "wind.speed": "wind_speed", "wind.deg": "wind_deg", 
                    "wind.gust": "wind_gust", "dt_txt": "forecast_date", "main.temp_kf": "temp_kf", "clouds.all": "clouds"}
    df = pd.json_normalize(list_data)\
    .drop(["weather", "pop", "dt", "visibility", "sys.pod", "main.sea_level", "main.grnd_level"], axis = 1)\
    .rename(dict_rename, axis = "columns")
    return df

df_fw = last_forecast_weather_to_df(client)

df_fw.head()


,forecast_date,windfarm_id,temp,feels_like,temp_min,temp_max,pressure,humidity,temp_kf,clouds,wind_speed,wind_deg,wind_gust
0,2023-06-13 09:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,22.31,21.80,22.31,23.40,1011,46,-1.09,38,5.53,76,7.74
1,2023-06-13 12:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,23.15,22.62,23.15,24.82,1011,42,-1.67,32,6.46,72,7.68
2,2023-06-13 15:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,24.06,23.46,24.06,24.93,1010,36,-0.87,13,5.73,72,7.40
3,2023-06-13 18:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,21.70,21.36,21.70,21.70,1010,55,0.00,0,2.88,51,4.67
4,2023-06-13 21:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,15.63,14.82,15.63,15.63,1012,60,0.00,0,3.44,55,5.33


In [49]:
def power_curve(eng) -> pd.DataFrame:
    df = pd.read_sql(
        """SELECT p.windturbine_id, windspeed, power, windfarm_id,latitude,longitude
            FROM mariadb_itw.windturbines as wt inner join mariadb_itw.powercurves as p
            ON p.windturbine_id = wt.windturbine_id ;""",
        con=eng
        )
    return df

eng = mariadb_connection()
df_pc = power_curve(eng)

In [61]:
def forecast_power_by_turbine(df_forecast_weather: pd.DataFrame, df_power_curve: pd.DataFrame) -> pd.DataFrame:

    # Get estimated power for the 5-next days based on forecast wind and powercurve
    df_final = pd.DataFrame()
    for index, row in df_forecast_weather.iterrows():
        df = df_power_curve[(df_power_curve["windfarm_id"] == row["windfarm_id"]) & (df_power_curve["windspeed"] == round(row["wind_speed"], 1))]
        df_copy = df.copy()
        df_copy.loc[:,"forecast_date"] = row["forecast_date"]
        df_final = pd.concat([df_final, df_copy])
    df_final.rename(columns = {"power": "power_kw"}, inplace = True)
    df_final.drop(columns = ["latitude", "longitude"], inplace=True)
    df_final.sort_values(by=["forecast_date", "windfarm_id", "windturbine_id"], inplace=True)
    df_final["prod_kwh"] = df_final["power_kw"] * 3
    df_final.reset_index(inplace=True, drop=True)
    return df_final

df_fp = forecast_power_by_turbine(df_fw, df_pc).groupby(["forecast_date", "windfarm_id"])["power_kw"].sum().reset_index()
df_fp

,forecast_date,windfarm_id,power_kw
0,2023-06-13 09:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,2816.0
1,2023-06-13 09:00:00,1ec6d7a1-4bf4-6ab6-a587-df2d2b033685,1639.2
2,2023-06-13 09:00:00,1ec6d7a1-4bf8-62b0-ac1e-df2d2b033685,4318.8
3,2023-06-13 09:00:00,1ec6d7a1-4bfb-6adc-9f35-df2d2b033685,1232.4
4,2023-06-13 09:00:00,1ec6d7a1-4bff-615a-92af-df2d2b033685,1428.6
...,...,...,...
475,2023-06-18 06:00:00,1ec6d7a1-4c09-6060-8336-df2d2b033685,0.0
476,2023-06-18 06:00:00,1ec88ba0-069f-6e5c-87cb-d1675f9a49d0,0.0
477,2023-06-18 06:00:00,1ec88ba1-ed6f-66c4-9472-13ed6e0d0396,0.0
478,2023-06-18 06:00:00,1ec88ba2-613a-6afe-81a1-a3486c5d0509,0.0


In [103]:
df_wf_turbine_id = pd.read_sql(
        """SELECT *
            FROM mariadb_itw.windturbines;""",
        con=eng
        )

df = df_wf_turbine_id[["latitude", "longitude"]].mean().iloc[0]
df = df_wf_turbine_id.groupby("windfarm_id")[["latitude", "longitude"]].mean()
df = df.reset_index()

df

,windfarm_id,latitude,longitude
0,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.721750,4.466809
1,1ec6d7a1-4bf4-6ab6-a587-df2d2b033685,48.659950,3.972095
2,1ec6d7a1-4bf8-62b0-ac1e-df2d2b033685,49.422175,4.549736
3,1ec6d7a1-4bfb-6adc-9f35-df2d2b033685,48.632617,4.184223
4,1ec6d7a1-4bff-615a-92af-df2d2b033685,48.615000,4.151485
5,1ec6d7a1-4c02-6594-9f23-df2d2b033685,48.616373,4.153123
6,1ec6d7a1-4c05-6b7c-aa3f-df2d2b033685,48.723588,4.414330
7,1ec6d7a1-4c09-6060-8336-df2d2b033685,49.779475,2.770939
8,1ec88ba0-069f-6e5c-87cb-d1675f9a49d0,48.649109,4.063658
9,1ec88ba1-ed6f-66c4-9472-13ed6e0d0396,48.628917,4.191688


In [106]:
df_wf_turbine = pd.read_sql(
        """SELECT *
            FROM mariadb_itw.windturbines;""",
        con=eng
        )

df = df_wf_turbine[["latitude", "longitude"]].mean()
    
print(df.iloc[0])
print(df.iloc[1])

48.82173272727273
4.099458727272728
